In [2]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:4 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:5 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:6 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:7 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:8 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:9 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease [21.3 kB]
Ign:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:11 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic/main Sources [1,814 kB]
Ign:12 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:13 https://developer.download

In [3]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2021-12-05 23:25:50--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  1.69MB/s    in 0.6s    

2021-12-05 23:25:52 (1.69 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [4]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

### Load Amazon Data into Spark DataFrame

In [5]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Tools_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Tools_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   15785389|R2UM5QMHBHC90Q|B00H5U9ZD6|     115362950|WallPeg 12 sq ft ...|           Tools|          5|            0|          0|   N|                Y|Great organizer, ...|Very nice. Will o...| 2015-08-31|
|         US|   47910848|  RF0D1LEIF6L7|B001TJGCS0|     570955425|Nite Ize Nite Daw...|           Tools|          4|    

### Create Vine DataFrame

In [11]:
from pyspark.sql.functions import to_date, count
from pyspark.sql.types import IntegerType


In [12]:
# Create the vine_table. DataFrame
vine_df = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
vine_df = vine_df.withColumn("star_rating", vine_df["star_rating"].cast(IntegerType()))
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2UM5QMHBHC90Q|          5|            0|          0|   N|                Y|
|  RF0D1LEIF6L7|          4|            0|          0|   N|                Y|
|  RM6YKIWQVNSY|          1|            6|          6|   N|                Y|
|R1RL3L68ASPS36|          4|            0|          0|   N|                Y|
|R1U4XFBFAG34CY|          5|            0|          0|   N|                Y|
|R3KFIK8P0I91PL|          5|            0|          0|   N|                Y|
| RENOAY76PPK1O|          5|            0|          0|   N|                Y|
| RINV884I0NL5V|          1|            0|          0|   N|                Y|
| R5KJH6CXZH2PX|          5|            0|          0|   N|                Y|
| RO69JF6QWD0W1|          5|            0|          0|   N|     

# Review Vine table for bias

In [13]:
# Extract only those reviews that have 20 or more votes
useful_reviews_df = vine_df.filter("total_votes>20")
useful_reviews_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RXAHWIC1584UQ|          5|           20|         23|   N|                Y|
| RSBELAIC899DO|          5|           46|         51|   N|                Y|
|R2HCC4CJ59D225|          4|           43|         43|   N|                Y|
|R39BVCCVPRV6F5|          5|           10|         22|   N|                Y|
| RL9VF9WXHEHKR|          5|           35|         36|   N|                Y|
|R1ZINLWUOLTZ46|          1|           39|         45|   N|                N|
|R1CQKM1K1CHOHS|          1|           21|         22|   N|                N|
|R1ZG19X1NOLCRF|          5|            5|         21|   N|                Y|
|R1YOX5Z4GF3ZSW|          1|            1|         22|   N|                Y|
| R2DHAM7J1KMWN|          5|           23|         24|   N|     

In [14]:
# Get reviews where helpful_votes make up more than half of the total votes
mostly_helpful_df = useful_reviews_df.filter("helpful_votes/total_votes>=0.5")
mostly_helpful_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RXAHWIC1584UQ|          5|           20|         23|   N|                Y|
| RSBELAIC899DO|          5|           46|         51|   N|                Y|
|R2HCC4CJ59D225|          4|           43|         43|   N|                Y|
| RL9VF9WXHEHKR|          5|           35|         36|   N|                Y|
|R1ZINLWUOLTZ46|          1|           39|         45|   N|                N|
|R1CQKM1K1CHOHS|          1|           21|         22|   N|                N|
| R2DHAM7J1KMWN|          5|           23|         24|   N|                N|
|R26EZ2INBETPU2|          1|          103|        128|   N|                N|
|R265ZMDO2ISN1O|          5|           20|         21|   N|                Y|
|R3NZ7JTOCPYE1S|          4|           32|         33|   N|     

In [15]:
# Now get the reviews that were part of the Vine program
vine_reviews_df = mostly_helpful_df.filter("vine=='Y'")
vine_reviews_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3QIQRG107AP19|          5|           38|         41|   Y|                N|
|R1ZNBHGCYE8VN1|          5|          175|        181|   Y|                N|
|R1BNZJ9IKGRBC2|          4|           63|         70|   Y|                N|
|R2IXTN4PBRHBO2|          5|           19|         24|   Y|                N|
|R32AOW3L09SYMQ|          5|           29|         34|   Y|                N|
|R34Z4VTLF7PC6X|          4|           33|         36|   Y|                N|
|R25403NE9JCRZZ|          2|           36|         40|   Y|                N|
| RDUP15OURNUSA|          5|           22|         23|   Y|                N|
|R1EI4NZTG7G14F|          5|           17|         23|   Y|                N|
|R2G0YB0MC67NLG|          5|           29|         38|   Y|     

In [16]:
# Reviews that were not part of the Vine program
not_vine_reviews_df = mostly_helpful_df.filter("vine=='N'")
not_vine_reviews_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RXAHWIC1584UQ|          5|           20|         23|   N|                Y|
| RSBELAIC899DO|          5|           46|         51|   N|                Y|
|R2HCC4CJ59D225|          4|           43|         43|   N|                Y|
| RL9VF9WXHEHKR|          5|           35|         36|   N|                Y|
|R1ZINLWUOLTZ46|          1|           39|         45|   N|                N|
|R1CQKM1K1CHOHS|          1|           21|         22|   N|                N|
| R2DHAM7J1KMWN|          5|           23|         24|   N|                N|
|R26EZ2INBETPU2|          1|          103|        128|   N|                N|
|R265ZMDO2ISN1O|          5|           20|         21|   N|                Y|
|R3NZ7JTOCPYE1S|          4|           32|         33|   N|     

In [40]:
star_counts_df = vine_reviews_df.groupby("star_rating").agg({"*":"count"})
star_counts_df = star_counts_df.toPandas()
five_stars_count = star_counts_df.loc[star_counts_df['star_rating'] == 5,'count(1)'].values[0]
total_reviews = vine_reviews_df.count()
vine_pct_five_stars = 100*(five_stars_count / total_reviews)
print(f'There are {total_reviews} total reviews, and {five_stars_count} 5-star ratings for reviews that were part of the Vine program')
print(f'Vine program five star ratings percentage: {round(vine_pct_five_stars,1)}%')



There are 268 total reviews, and 149 5-star ratings for reviews that were part of the Vine program
Vine program five star ratings percentage: 55.6%


In [41]:
star_counts_df = not_vine_reviews_df.groupby("star_rating").agg({"*":"count"})
star_counts_df = star_counts_df.toPandas()
five_stars_count = star_counts_df.loc[star_counts_df['star_rating'] == 5,'count(1)'].values[0]
total_reviews = not_vine_reviews_df.count()
not_vine_pct_five_stars = 100*(five_stars_count / total_reviews)
print(f'There are {total_reviews} total reviews, and {five_stars_count} 5-star ratings for reviews that were part not of the Vine program')
print(f'Non-Vine program five star ratings percentage: {round(not_vine_pct_five_stars,1)}%')

There are 29517 total reviews, and 13675 5-star ratings for reviews that were part not of the Vine program
Non-Vine program five star ratings percentage: 46.3%
